In [60]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import matplotlib.pyplot as plt
import pandas as pd
from src.model import SeasonModel

plt.rcParams['figure.figsize'] = 15.7,13.27
plt.rcParams['font.family'] = 'AppleGothic'

class LEARN_INFO:
    def __init__(self, name, columns, is_contain_cluster_label=False):
        self.name = name
        self.columns = columns
        self.is_label = is_contain_cluster_label
    

# INPUT_REPORT_SIMULATION = [
#     LEARN_INFO(name="univariate", columns=['energy (kw 15min)'], is_contain_cluster_label=False),
#     LEARN_INFO(name="with cluster label", columns=['energy (kw 15min)'], is_contain_cluster_label=True),
# ]

DEFAULT_COLUMNS = [
    'energy (kw 15min)',
    'week sin',
    'week cos',
    'year sin',
    'year cos',
    'season idx',
    'weather idx',
    'avg ta',
    'avg rhm'
]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
sm = SeasonModel(name="season test",
                   columns=['energy (kw 15min)', 'season idx'],
                   is_contain_cluster_label=True,
                   is_switch=False)
sm

###### Basic Model Information ######
 - name : season test
 - columns : energy (kw 15min),season idx
 - datas : 
                     energy (kw 15min)  season idx
Date Time                                         
2017-01-01 00:00:00              15250           4
2017-01-01 01:00:00              14000           4
2017-01-01 02:00:00              14750           4
2017-01-01 03:00:00              29500           4
2017-01-01 04:00:00              15750           4
 - norm_datas : 
                     energy (kw 15min)  cluster energy
Date Time                                             
2017-03-01 00:00:00          -1.043057       -0.387447
2017-03-01 01:00:00          -1.175362       -0.423218
2017-03-01 02:00:00          -1.175362       -0.438591
2017-03-01 03:00:00          -0.271280       -0.201493
2017-03-01 04:00:00          -1.175362       -0.295505
                     energy (kw 15min)  cluster energy
Date Time                                             
2017-06-01 00:00:

In [55]:
_ = {
    "univariate": ['energy (kw 15min)'],
    "multivariate (week)": [
        'energy (kw 15min)',
        'week sin',
        'week cos'
    ],
    "multivariate (year)": [
        'energy (kw 15min)',
        'year sin',
        'year cos',
    ],
    "multivariate (weather)": [
        'energy (kw 15min)',
        'weather idx'
    ],
    "multivariate (avg ta)": [
        'energy (kw 15min)',
        'avg ta'
    ],
    "multivariate (avg rhm)": [
        'energy (kw 15min)',
        'avg rhm'
    ]
}

# Input Report Simulation
IRS = []
for key in _.keys():
    IRS.append(
        LEARN_INFO(name=key, 
                   columns=_[key], 
                   is_contain_cluster_label=False),
    )
    
for key in _.keys():
    IRS.append(
        LEARN_INFO(name="{} / with cluster label".format(key), 
                   columns=_[key], 
                   is_contain_cluster_label=True),
    )

# Training Model Array 
TMA = []

is_switch = False

for _ in IRS:
    sm = SeasonModel(name=_.name,
                   columns=_.columns,
                   is_contain_cluster_label=_.is_label,
                   is_switch=is_switch)
    
    sm.set_window()
    sm.set_model()
    print(sm)
    sm.training(epochs=50)
    
    sm.set_predict(is_reshape=True)
    sm.set_predict(is_reshape=True, is_val_datas=True)
    
    TMA.append(sm)
    
test_evaluate_df = pd.DataFrame();
val_evaluate_df = pd.DataFrame();

for _ in TMA:
    test_evaluate_df = test_evaluate_df.append(_.statistic(), ignore_index=True)
    val_evaluate_df = val_evaluate_df.append(_.statistic(is_val_datas=True), ignore_index=True)
    
test_evaluate_df.index = [_.name for _ in TMA]
val_evaluate_df.index = [_.name for _ in TMA]

###### [Notice] (multivariate (avg rhm) / with cluster label) set predict (validation) info start ###### 

                      avg rhm  energy (kw 15min)  cluster energy
Date Time                                                       
2018-03-01 00:00:00  0.862706          -2.255850      312.552738
2018-03-01 01:00:00  0.862706          -2.277900      304.619476
2018-03-01 02:00:00  0.862706          -2.299951      297.063988
2018-03-01 03:00:00  0.862706          -2.299951      490.862247
2018-03-01 04:00:00  0.862706          -2.277900      407.374108
...                       ...                ...             ...
2018-02-28 19:00:00  1.440279          -0.635227      772.967580
2018-02-28 20:00:00  1.440279          -0.770706      667.534849
2018-02-28 21:00:00  1.440279          -0.923119      602.011683
2018-02-28 22:00:00  1.440279          -0.990858      419.539593
2018-02-28 23:00:00  1.440279          -1.465033      367.121060

[2184 rows x 3 columns]
0 / 91
50 / 91
91 / 91 

In [56]:
test_evaluate_df

,ecv,mae,mean dis,mean sim,mse,tss,wss
univariate,89.428833,0.304917,1.797038,0.939187,0.176248,5987.118007,632.908246
multivariate (week),89.690836,0.297950,1.762043,0.939710,0.171880,5987.118007,617.221831
multivariate (year),89.687681,0.310327,1.803116,0.936609,0.171933,5987.118007,617.410737
multivariate (weather),89.234695,0.301657,1.799086,0.938120,0.179485,5987.118007,644.531505
multivariate (avg ta),88.315193,0.318225,1.867596,0.936906,0.194816,5987.118007,699.583198
multivariate (avg rhm),89.349310,0.304155,1.800936,0.938953,0.177574,5987.118007,637.669383
univariate / with cluster label,91.222703,0.274869,1.636548,0.940399,0.146340,5987.118007,525.507111
multivariate (week) / with cluster label,91.211047,0.274076,1.627172,0.941553,0.146534,5987.118007,526.204958
multivariate (year) / with cluster label,91.371766,0.276496,1.620537,0.945644,0.143855,5987.118007,516.582528
multivariate (weather) / with cluster label,90.890455,0.278588,1.663626,0.939311,0.151879,5987.118007,545.399194


In [57]:
val_evaluate_df

,ecv,mae,mean dis,mean sim,mse,tss,wss
univariate,74.355995,0.249343,1.531008,0.935014,0.126299,941.187941,241.358285
multivariate (week),75.864707,0.241579,1.490705,0.937145,0.118869,941.187941,227.158465
multivariate (year),68.745917,0.308568,1.715194,0.936641,0.153930,941.187941,294.159657
multivariate (weather),75.915579,0.239422,1.480389,0.935902,0.118618,941.187941,226.679670
multivariate (avg ta),74.882660,0.253384,1.526615,0.937429,0.123706,941.187941,236.401377
multivariate (avg rhm),74.687954,0.249377,1.516027,0.936809,0.124665,941.187941,238.233928
univariate / with cluster label,79.339454,0.223204,1.382107,0.944452,0.101755,941.187941,194.454565
multivariate (week) / with cluster label,79.728792,0.231692,1.374384,0.945685,0.099838,941.187941,190.790164
multivariate (year) / with cluster label,76.737344,0.257692,1.484059,0.940169,0.114571,941.187941,218.945316
multivariate (weather) / with cluster label,77.986853,0.225529,1.414671,0.941425,0.108417,941.187941,207.185089


In [58]:
test_evaluate_df.to_csv('path\\test_input_result.csv', sep=',', na_rep='NaN')

In [59]:
val_evaluate_df.to_csv('path\\val_input_result.csv', sep=',', na_rep='NaN')

In [ ]:
pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 100)